In [ ]:
# e1f5wbog0 - class for bbc anchor tags 
from urllib.request import urlopen
from datetime import datetime
from bs4 import BeautifulSoup
import psycopg2

In [ ]:
def get_html(url):
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode('utf_8')
    return html

In [ ]:
def parse_stories(domain_url, html):
    soup = BeautifulSoup(html, 'html.parser')
    anchor_tags = soup.select('.e1f5wbog0')
    stories = []
    for tag in anchor_tags:
        url = domain_url + tag.get('href')
        title = tag.get_text()
        if url and title:
            stories.append((title, url, datetime.utcnow(), datetime.utcnow()))

    return stories

In [ ]:
def connect_to_db():
    conn = psycopg2.connect(host="localhost",database="social_news", user="sonali")
    return conn

In [ ]:
def insert_stories(stories):
    sql = "INSERT INTO stories(title, url, created_at, updated_at) VALUES(%s, %s, %s, %s)"
    conn = None
    try: 
        conn = connect_to_db()
        cur = conn.cursor()
        cur.executemany(sql, stories)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: ")
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def get_stories():
    conn = None
    try:
        conn = connect_to_db()
        cur = conn.cursor()
        cur.execute('SELECT * FROM stories ORDER BY id DESC;')
        data = cur.fetchall()
        print(cur.rowcount)
        for row in data:
            print(row)
        
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def run():
    url = "http://bbc.co.uk"
    stories = parse_stories(url, get_html(url))
    # insert_stories(stories)
    # get_stories()

# run()